# 본문 추출 코드 
- txt파일 내용 추출/ 파일 이름에서 날짜 추출 -> txt 파일 내용으로 날짜 지정하여 검색 ->  가장 위에 있는 네이버 뉴스 기사의 본문 추출

In [6]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# 파일에서 제목을 추출하여 리스트에 저장하는 함수
def extract_titles(filename):
    lines = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            stripped_line = line.strip()
            if '. ' in stripped_line:
                text = stripped_line.split('. ', 1)[1]
                lines.append(text)
    return lines

# 파일 이름에서 날짜를 추출하여 변수에 저장하는 함수
def extract_date(filename):
    match = re.search(r'(\d{4})(\d{2})(\d{2})', filename)
    if match:
        year, month, day = match.groups()
        date = f'{year}.{month}.{day}'
        return date
    return None

# 뉴스 본문을 수집하는 함수
def get_first_news_body(query, date):
    search_url = f"https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{date.replace('.','')}to{date.replace('.','')}&is_sug_officeid=0&office_category=0&service_area=0"
    search_response = requests.get(search_url)
    search_response.raise_for_status()
    search_soup = BeautifulSoup(search_response.text, 'html.parser')
    
    for i in range(1, 10):  
        try:
            news_link = search_soup.select('.info_group a')[i]['href']
            news_response = requests.get(news_link)
            news_response.raise_for_status()
            news_soup = BeautifulSoup(news_response.text, 'html.parser')
            news_body_element = news_soup.select_one('#dic_area')
            if news_body_element:
                return news_body_element.get_text(strip=True)
        except (IndexError, ValueError, requests.exceptions.RequestException):
            continue
    return "본문을 찾을 수 없습니다."

# 데이터 수집 범위 설정
start_date = '20230701'
end_date = '20240701'
start_dt = datetime.strptime(start_date, '%Y%m%d')
end_dt = datetime.strptime(end_date, '%Y%m%d')

data = []

# 날짜 범위 내에서 파일 이름 생성 및 데이터 수집
for n in range(int((end_dt - start_dt).days) + 1):
    current_date = (start_dt + timedelta(n)).strftime('%Y%m%d')
    file_path = f"이슈_검색어/important_news_advice_{current_date}.txt"
    
    try:
        # 파일에서 제목 추출
        queries = extract_titles(file_path)
        date = extract_date(file_path)
        
        # 각 제목에 대해 뉴스 본문 수집
        for query in queries:
            news_body = get_first_news_body(query, date)
            data.append({'검색어': query, '본문': news_body, '날짜': date})
    except FileNotFoundError:
        print(f"{file_path} 파일을 찾을 수 없습니다.")

# 데이터프레임 생성
df = pd.DataFrame(data)

# 파일 저장

In [62]:
df.to_csv('important_news_advice_content.csv',index=False)